# University League Table generator --- your university can be the best!

Python Jupyter noteobook to read (slightly tidied) data set for 2019 Guardian University league table, and then use simulated annealing to try and make a University you select have the highest leaue table position

Cell just below imports the usual plus google.colab which you need to have to read in the xlsx - which in your case you will need to save to your Google Drive

Written by Richard Sear www.richardsear.me r.sear@surrey.ac.uk
Do anything you like with it, just don't blame me.

In [11]:
import numpy as np
import pandas as pd
#from google.colab import drive
#drive.mount('/content/drive/')
############################
import scipy.optimize
import time

Cell below reads in xlsx into a Pandas dataframe

In [12]:
file_location = r'/content/drive/My Drive/Colab Notebooks/Guardian_University_Guide_2019edited.xlsx'
file_location = r'Guardian_University_Guide_2019edited.xlsx'
file_location = r'Guardian_University_Guide_2019editedV2.xlsx'

#df = pd.read_excel(file_location)
xl = pd.ExcelFile(file_location)
print(xl.sheet_names)  # see all sheet names

df=xl.parse("Institution")  # read a specific sheet to DataFrame
print("Column headings:")
print(df.columns)

['Institution', 'S010 Med', 'S020 Dent', 'S030 Vet', 'S040 Anatomy', 'S050 Nursing', 'S060 Social', 'S070 Health', 'S080 Psych', 'S090 Pharm', 'S100 Bio', 'S110 Chem', 'S120 ForArch', 'S130 Physics', 'S140 Earth', 'S150 Agri', 'S160 GenE', 'S170 ChemE', 'S180 MatE', 'S190 MechE', 'S200 CivilE', 'S210 ElecE', 'S220 IT', 'S230 Maths', 'S240 BusMan', 'S250 AccFin', 'S260 Tourism', 'S270 Geog', 'S280 SocPol', 'S290 Ed', 'S300 Law', 'S310 Socio', 'S315 Crim', 'S320 Anth', 'S330 America', 'S340 History', 'S345 HofArt', 'S350 Pol', 'S360 Econ', 'S370 Lang', 'S380 English', 'S390 Media', 'S395 Journ', 'S400 Art', 'S410 Design', 'S415 Fash', 'S420 Drama', 'S430 Film', 'S440 Music', 'S450 Arch', 'S460 Class', 'S470 Phil', 'S480 Theo', 'S490 Sports', 'S500 Plan']
Column headings:
Index(['Ranking', 'Unnamed: 1', 'Unnamed: 2', 'Ranking.1', 'Institution',
       'Average Teaching Score', 'satisfied with teaching (%)',
       'satisfied with course (%)', 'Expenditure per student / 10',
       'Studen

In [13]:
#number of universities
n_uni=121
#n_uni=100
print('running for top ',n_uni,' universities only NB sim ann code is crude so struggles with all 100 + unis')
'''
in scores_df ROWS are universities
column 0 is uni name
column 1 (Average Teaching Score) is where we put the ranking
columns 2 to 10 are the 9 metrics used to assess unis
'''
# reads in desired set of unis into a dataframe
scores_df=df.iloc[0:n_uni,4:16]
print('Guardian League Table is:')
for i_uni in range(0,n_uni):
  print('{:>27}'.format(scores_df.iloc[i_uni,0]),' ranked ','{:3d}'.format(i_uni+1),' with score ','{:6.2f}'.format(scores_df.iloc[i_uni,1]),i_uni)

running for top  121  universities only NB sim ann code is crude so struggles with all 100 + unis
Guardian League Table is:
                  Cambridge  ranked    1  with score  100.00 0
                     Oxford  ranked    2  with score   97.40 1
                 St Andrews  ranked    3  with score   97.20 2
               Loughborough  ranked    4  with score   87.90 3
                     Durham  ranked    5  with score   87.60 4
                       Bath  ranked    6  with score   84.10 5
           Imperial College  ranked    7  with score   83.30 6
                    Warwick  ranked    8  with score   80.90 7
                  Lancaster  ranked    9  with score   80.20 8
                      Leeds  ranked   10  with score   78.80 9
                        UCL  ranked   11  with score   78.00 10
                       York  ranked   12  with score   77.90 11
                   Coventry  ranked   13  with score   77.80 12
                     Exeter  ranked   14  with score  

In [24]:
# this finds rank of desired uni given a weights vector
def rank_uni(weights):
    for i_uni in range(0,n_uni):
        score_vector=weights[:]*scores_df.iloc[i_uni,2:2+n]
        scores_df.iloc[i_uni,1]=np.sum(score_vector)
# sort
    sorted_df=scores_df.sort_values(by=['Average Teaching Score'],ascending=False)
# find rank
#    print('score of uni currently top ',sorted_df.iloc[0,0],sorted_df.iloc[0,1])
    for i_uni in range(0,n_uni):
        if(sorted_df.iloc[i_uni,0]==uni_name):
            n_rank=i_uni+1
#            print('score of uni we want top ',sorted_df.iloc[i_uni,0],sorted_df.iloc[i_uni,1])
            break
    dmin=-sorted_df.iloc[n_rank-1,1]+sorted_df.iloc[0,1]
#    print('difference in scores ',dmin)
    return n_rank,sorted_df,dmin

def fmin(weights):
    n_rank,sorted_df,dmin=rank_uni(weights)
#    print(n_rank,weights,dmin)
    return dmin

# number of inputs used, there are 9 numbers per University
n=9
scores_df['Average Teaching Score'] = np.zeros(n_uni)

In [22]:

# select a particular Uni - JUST CHANGE THIS TO TRY AND MAKE ANOTHER UNI THE BEST

#

#
for ij in range(0,n_uni):
    uni_name=scores_df.iloc[ij,0]
    print('uni ',uni_name)
    t_start=time.perf_counter()
    for i_try in range(0,30):
# first guess at League Table weights
        LTweights=np.random.normal(loc=1.5-float(ij)/n_uni,scale=2.0*float(ij),size=n)
# get an initial ranking
        initial_rank,sorted_df,diff=rank_uni(LTweights)
        print('initial rank is ',initial_rank)
        out=scipy.optimize.minimize(fmin,LTweights,method='BFGS')#,method='Nelder-Mead')
        final_LTweights=out.x
        final_rank,final_df,diff=rank_uni(final_LTweights)
        if(final_rank==1): 
            break
        else:
            print('failed ',i_try)
#
    t_end=time.perf_counter()
    t_min=t_end-t_start
    print('took ',t_min/60.0,' minutes')
#print(out)
    print(out.x)
#print(final_LTweights)

    print('')
    print('')
    print(uni_name,' has highest league table position of ',final_rank)
    final_weights=np.round(final_LTweights,3)
    print('values of weights that give this league table ',final_weights)
    if(final_rank==1):
        print('Congratulations! ',uni_name,' has highest league table position of ',final_rank)
    else:
        print('Sorry League Table Omatic has failed you ',uni_name,' has highest league table position of ',final_rank)
#
    f = open("top_weights.txt", "a")
    stringy=uni_name+' '
    for i in range(0,9):
        stringy += str(np.round(final_LTweights[i],5))+' '
    stringy += str(n_uni)+ '\n'
    f.write(stringy)
    f.close()

uni  Cambridge
initial rank is  1
1 [1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5] 0.0
1 [1.50000001 1.5        1.5        1.5        1.5        1.5
 1.5        1.5        1.5       ] 0.0
1 [1.5        1.50000001 1.5        1.5        1.5        1.5
 1.5        1.5        1.5       ] 0.0
1 [1.5        1.5        1.50000001 1.5        1.5        1.5
 1.5        1.5        1.5       ] 0.0
1 [1.5        1.5        1.5        1.50000001 1.5        1.5
 1.5        1.5        1.5       ] 0.0
1 [1.5        1.5        1.5        1.5        1.50000001 1.5
 1.5        1.5        1.5       ] 0.0
1 [1.5        1.5        1.5        1.5        1.5        1.50000001
 1.5        1.5        1.5       ] 0.0
1 [1.5        1.5        1.5        1.5        1.5        1.5
 1.50000001 1.5        1.5       ] 0.0
1 [1.5        1.5        1.5        1.5        1.5        1.5
 1.5        1.50000001 1.5       ] 0.0
1 [1.5        1.5        1.5        1.5        1.5        1.5
 1.5        1.5        1.50000001] 0.0
took  

1 [ 2.15207342  1.16649633  0.77975947  3.96176841  1.05686272  0.82304296
  4.58812581  0.28524568 -0.31226659] 0.0
1 [ 2.15207342  1.16649633  0.77975947  3.96176839  1.05686274  0.82304296
  4.58812581  0.28524568 -0.31226659] 0.0
1 [ 2.15207342  1.16649633  0.77975947  3.96176839  1.05686272  0.82304298
  4.58812581  0.28524568 -0.31226659] 0.0
1 [ 2.15207342  1.16649633  0.77975947  3.96176839  1.05686272  0.82304296
  4.58812582  0.28524568 -0.31226659] 0.0
1 [ 2.15207342  1.16649633  0.77975947  3.96176839  1.05686272  0.82304296
  4.58812581  0.2852457  -0.31226659] 0.0
1 [ 2.15207342  1.16649633  0.77975947  3.96176839  1.05686272  0.82304296
  4.58812581  0.28524568 -0.31226658] 0.0
took  0.08046999068319564  minutes
[ 2.15207342  1.16649633  0.77975947  3.96176839  1.05686272  0.82304296
  4.58812581  0.28524568 -0.31226659]


Oxford  has highest league table position of  1
values of weights that give this league table  [ 2.152  1.166  0.78   3.962  1.057  0.823  4.588  0.28

KeyboardInterrupt: 

Now finally just print out the league table, with scores

In [6]:
for i_uni in range(0,n_uni):
  print('{:>27}'.format(final_df.iloc[i_uni,0]),' ranked ','{:3d}'.format(i_uni+1),' with score ','{:6.2f}'.format(final_df.iloc[i_uni,1]))

       Bucks New University  ranked    1  with score  -35148.13
                Westminster  ranked    2  with score  -35790.51
                    Cumbria  ranked    3  with score  -35886.08
              Hertfordshire  ranked    4  with score  -35907.25
          London South Bank  ranked    5  with score  -35979.85
                De Montfort  ranked    6  with score  -36051.45
                 London Met  ranked    7  with score  -36064.66
                   Bradford  ranked    8  with score  -36966.24
                   Brighton  ranked    9  with score  -37318.07
               Bedfordshire  ranked   10  with score  -37330.15
                 Roehampton  ranked   11  with score  -37743.95
              Leeds Trinity  ranked   12  with score  -38264.92
                     Brunel  ranked   13  with score  -38311.33
                   Kingston  ranked   14  with score  -38458.67
                       Hull  ranked   15  with score  -38659.81
                   Teesside  ranked   16

In [33]:
#from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution
#
uni_name=scores_df.iloc[69,0]
print('uni ',uni_name)
# first guess at League Table weights
LTweights=np.random.normal(loc=1.5,scale=2.0,size=n)
# get an initial ranking
initial_rank,sorted_df,diff=rank_uni(LTweights)
print('initial rank is ',initial_rank)
#out=scipy.optimize.minimize(fmin,LTweights,method='BFGS')#,method='Nelder-Mead')
bounds = [[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]]
#out=dual_annealing(fmin,bounds)#,method='Nelder-Mead')
out=differential_evolution(fmin,bounds)#,method='Nelder-Mead')

print(out)
final_LTweights=out.x
final_rank,final_df,diff=rank_uni(final_LTweights)
if(final_rank==1):
    print('Congratulations! ',uni_name,' has highest league table position of ',final_rank)
else:
    print('Sorry League Table Omatic has failed you ',uni_name,' has highest league table position of ',final_rank)
#

uni  Sheffield Hallam
initial rank is  69
     fun: 8.562023726348784e-07
     jac: array([ 1.69004979,  1.95138057,  0.43902935,  2.07630816,  0.07094853,
       -0.11979342, -0.18181359,  1.70140344,  2.41923557])
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 135705
     nit: 1000
 success: False
       x: array([ 1.69844621e-07,  6.13975006e-07,  2.43347355e-06,  6.01071407e-07,
       -5.05522032e-07,  1.44474264e-06,  1.37600779e-06, -2.51171030e-07,
       -6.33658829e-07])
Sorry League Table Omatic has failed you  Sheffield Hallam  has highest league table position of  6
